# Revisiting code

In [1]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake
from shared_utils import geography_utils
import shared_utils
from plotnine import *
import altair as alt
import altair_saver
from shared_utils import altair_utils 
import cpi

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
#NB ONLY
import data_prep_refactored
import data_prep 
import agency_crosswalk
pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 250

In [3]:
#Load in data
df_5311 = data_prep_refactored.load_grantprojects()
vehicles = data_prep_refactored.clean_vehicles_data()
organizations = data_prep_refactored.load_cleaned_organizations_data()
airtable = data_prep_refactored.load_airtable()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Testing script

In [4]:
#New script
test_aggregate = data_prep_refactored.aggregated_df()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
test_aggregate['reporter_type'].value_counts()

Rural Reporter        49
Full Reporter         26
None                   9
Reduced Reporter       2
Asset Subrecipient     1
Name: reporter_type, dtype: int64

In [6]:
test_aggregate['Is_Agency_In_BC_Only_1_means_Yes'].value_counts()

0    78
1     9
Name: Is_Agency_In_BC_Only_1_means_Yes, dtype: int64

In [7]:
test_aggregate['GTFS'].value_counts()

Static OK_RT Incomplete            54
Static OK_RT OK                    15
None                               11
Static Incomplete_RT Incomplete     5
Static Incomplete_RT OK             2
Name: GTFS, dtype: int64

In [8]:
test_aggregate['caltrans_district'].value_counts()

None                    17
06 - Fresno             16
04 - Oakland             9
03 - Marysville          8
05 - San Luis Obispo     7
08 - San Bernardino      7
10 - Stockton            6
01 - Eureka              6
02 - Redding             4
09 - Bishop              3
11 - San Diego           3
07 - Los Angeles         1
Name: caltrans_district, dtype: int64

In [24]:
test_aggregate.shape

(87, 38)

In [9]:
#test_aggregate.groupby(['fleet_size','organization_name']).agg({'total_vehicles':'sum'})

In [10]:
original = data_prep.aggregated_merged_df()

/home/jovyan/data-analyses/5311/data_prep.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [11]:
original['reporter_type'].value_counts()

Rural Reporter        49
Full Reporter         26
None                   9
Reduced Reporter       2
Asset Subrecipient     1
Name: reporter_type, dtype: int64

In [12]:
original['caltrans_district'].value_counts()

06 - Fresno             13
03 - Marysville          7
04 - Oakland             7
08 - San Bernardino      7
05 - San Luis Obispo     6
01 - Eureka              5
10 - Stockton            4
09 - Bishop              3
11 - San Diego           3
02 - Redding             3
07 - Los Angeles         1
Name: caltrans_district, dtype: int64

In [13]:
original['GTFS_Status'].value_counts()

Only Static            45
None                   27
Both static & RT OK    13
Only RT                 2
Name: GTFS_Status, dtype: int64

In [14]:
'''
original.to_csv(f"./original.csv", index=False)
test_aggregate.to_csv(f"./test.csv", index=False)
'''


'\noriginal.to_csv(f"./original.csv", index=False)\ntest_aggregate.to_csv(f"./test.csv", index=False)\n'

In [15]:
original.shape

(87, 44)

In [16]:
original.head(2)

,organization_name,allocationamount_mean,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,reporter_type,fleet_size,ntd_id,itp_id,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,GTFS_Status
0,Alpine County Community Development,48766.57,341366.00,293366.00,159901.31,181464.69,0,363613.49,173611.38,315613.45,190002.10,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,None,No Info,NaN,0,NaN,NaN,NaN,NaN,NaN,None
1,Amador Regional Transit System,189817.38,1518539.00,1279114.00,1067984.00,450555.00,0,1612119.29,1151645.28,1372694.08,460474.01,0.00,24.00,4.07,0.00,0.00,0.00,0.00,0.00,21.00,0.00,0.00,21.00,0.00,21.00,18.00,3.00,1.00,1.00,1.00,0.00,0.00,2.00,0.00,Rural Reporter,Medium,9R02-91000,11.00,10 - Stockton,NaN,Amador County Transportation Commission,Static OK,RT Incomplete,Only Static


In [17]:

set(original.columns).difference(set(test_aggregate.columns))

{'GTFS_Status',
 '_0_9',
 '_10_12',
 '_13_15',
 '_16_20',
 '_21_25',
 '_26_30',
 'allocationamount_mean',
 'gtfs_realtime_status',
 'gtfs_static_status'}

In [18]:
org_original = set(original.organization_name.unique().tolist())
org_script = set(test_aggregate.organization_name.unique().tolist())

In [19]:
org_original-org_script

set()

In [20]:
org_script-org_original

set()

In [21]:
def pick_column_and_aggregate(df1, df2, col):
    if df1[col].sum() == df2[col].sum():
        print("PASS")
    else:
        print(f"{col}: FAIL")

In [22]:
my_columns = ['allocationamount',
       'encumbered_amount', 'expendedamount', 'activebalance',
       'closedoutbalance', 'adjusted_allocationamount',
       'adjusted_expendedamount', 'adjusted_encumbered_amount',
       'adjusted_activebalance', 'adjusted_closedoutbalance', 'total_vehicles',]

In [23]:
for c in my_columns:
    pick_column_and_aggregate(test_aggregate, original, c)

PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
